<a href="https://colab.research.google.com/github/hmtu14/BookP/blob/master/gemma2_smallcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install datasets
# !pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 31.8 MB/s eta 0:00:00


In [2]:
%cd wsdm
%load_ext commonlib.utils
from commonlib.utils import initialize_experiment

/content/wsdm


In [3]:
initialize_experiment(
    name="gemma2_smallcv",
    version="",
    gpus=[0],
)
# Import from commonlib
from commonlib.header import *


2024-12-16, 11:24:56 - The default behavior of function 'print' has been overrided.
2024-12-16, 11:24:56 - Set EXPERIMENT_NAME to gemma2_smallcv
2024-12-16, 11:24:56 - Set PATH.COMMON to /content/wsdm
2024-12-16, 11:24:56 - Set PATH.DRIVE to /content/drive/MyDrive/kaggle/wsdm/gemma2_smallcv
2024-12-16, 11:24:56 - Set PATH.DATA to /content/drive/MyDrive/kaggle/wsdm/data
2024-12-16, 11:24:56 - Set PATH.LOGS to /content/drive/MyDrive/kaggle/wsdm/logs/gemma2_smallcv
2024-12-16, 11:24:56 - Set PATH.OOF to /content/drive/MyDrive/kaggle/wsdm/oof/gemma2_smallcv
2024-12-16, 11:24:56 - Set PATH.MODELS to /content/drive/MyDrive/kaggle/wsdm/models/gemma2_smallcv
2024-12-16, 11:24:56 - Set PATH.BACKBONE_MODELS to /content/drive/MyDrive/kaggle/wsdm/backbone_models
/content/wsdm


In [4]:
from datasets import Dataset
from datasets import load_dataset
import transformers
import torch
import pandas as pd
import numpy as np
import random
import os
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    MT5ForSequenceClassification,
    MT5TokenizerFast,
    MT5Config,
    Qwen2ForSequenceClassification,
    Qwen2TokenizerFast,
    Qwen2Config,
    PreTrainedTokenizerBase,
    EvalPrediction,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    set_seed
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import bitsandbytes
from dataclasses import dataclass
# Miscellaneous
import warnings
import keras
from sklearn.metrics import log_loss, accuracy_score
warnings.filterwarnings('ignore')

In [5]:
def set_all_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    set_seed(seed)
    # For TensorFlow (if used)
    # try:
    #     import tensorflow as tf
    #     tf.random.set_seed(seed)
    #     tf.config.experimental.enable_op_determinism()
    # except ImportError:
    #     pass
    # Ensure deterministic behavior in PyTorch
    # torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_all_seeds()

In [6]:
# print(tf.__version__)
print(keras.__version__)
print(transformers.__version__)
print(torch.__version__)
print(torch.version.cuda)

2024-12-16, 11:25:14 - 3.5.0
2024-12-16, 11:25:14 - 4.46.3
2024-12-16, 11:25:14 - 2.5.1+cu121
2024-12-16, 11:25:14 - 12.1


In [7]:
@dataclass
class Config:
    output_dir: str = PATH.MODELS
    checkpoint: str = os.path.join(
        PATH.BACKBONE_MODELS,
        "models--unsloth--gemma-2-9b-it-bnb-4bit",
        "snapshots",
        "27b027bcbb6b1861b02551d5c699d5d07f29610a"
    )  # 4-bit quantized gemma-2-9b-instruct
    # checkpoint: str = "Qwen/Qwen2-0.5B"
    max_length: int = 1024
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 24
    gradient_accumulation_steps: int = 1  # global batch size is 8
    per_device_eval_batch_size: int = 24
    n_epochs: int = 1
    # freeze_layers: int = 16  # there're 42 layers in total, we don't add adapters to the first 16 layers
    freeze_layers: int = 0
    lr: float = 1e-4
    warmup_steps: int = 120
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"

    def __post_init__(self):
        # Automatically set per_device_eval_batch_size to 2 * per_device_train_batch_size if not specified
        if self.per_device_eval_batch_size is None:
            self.per_device_eval_batch_size = 2 * self.per_device_train_batch_size

config = Config()

# Model

In [8]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=[
        # "q", "k", "v", "o",
        "q_proj", "k_proj", "v_proj", "o_proj",
        # "gate_proj", "up_proj", "down_proj"
    ],
    # layers_to_transform=[i for i in range(24) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)


In [9]:
# tokenizer = Qwen2TokenizerFast.from_pretrained(config.checkpoint)
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

In [10]:
%%time
# model = Qwen2ForSequenceClassification.from_pretrained(
#     config.checkpoint,
#     num_labels=2,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=2,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
# model.config.pad_token_id = model.config.eos_token_id
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/kaggle/wsdm/backbone_models/models--unsloth--gemma-2-9b-it-bnb-4bit/snapshots/27b027bcbb6b1861b02551d5c699d5d07f29610a and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 6.95 s, sys: 3.06 s, total: 10 s
Wall time: 1min 1s


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-41): 42 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3584, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDic

In [11]:
model.print_trainable_parameters()


trainable params: 17,898,496 || all params: 9,259,611,648 || trainable%: 0.1933


In [12]:
# (score): ModulesToSaveWrapper(
#         (original_module): Linear(in_features=3584, out_features=2, bias=False)
#         (modules_to_save): ModuleDict(
#           (default): Linear(in_features=3584, out_features=2, bias=False)
#         )
#       )

# Dataset

In [13]:
df_train = pd.read_csv(os.path.join(PATH.DATA, "train.csv"))
df_fold = pd.read_csv(os.path.join(PATH.DATA, "fold.csv"))
df_train = df_train.merge(df_fold, on="id", how="inner")

In [14]:
# ds = Dataset.from_csv("train.csv")
train_ds = Dataset.from_pandas(df_train.query("fold > 5").query("fold <= 20"))
# eval_ds = Dataset.from_pandas(df_train.query("fold == 0"))
# train_ds = Dataset.from_pandas(df_train.query("fold >= 5")).select(range(10))
eval_ds = Dataset.from_pandas(df_train.query("fold < 5"))

print(len(train_ds))
print(len(eval_ds))



2024-12-16, 11:26:27 - 14535
2024-12-16, 11:26:27 - 4845


In [15]:
class CustomTokenizer:
    def __init__(self, tokenizer: PreTrainedTokenizerBase, max_length: int) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        # Ensure that the keys exist in the batch before processing
        prompt = ["<prompt>: " + self.process_text(t) for t in batch.get("prompt", [])]
        # metadata
        # metadata_1 = ["\n\n<note>: response_a is generated by " + self.process_text(t) for t in batch.get("model_a", [])]
        # metadata_2 = ["<note>: response_b is generated by " + self.process_text(t) for t in batch.get("model_b", [])]
        # metadata_3 = ["<note>: the language is " + self.process_text(t) for t in batch.get("language", [])]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch.get("response_a", [])]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch.get("response_b", [])]

        # Concatenate all parts into one text field for tokenization
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]

        # Tokenize the texts
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True, padding=True)

        # Handle the winner labels (mapping winner from 'model_a' to 0, 'model_b' to 1)
        labels = []
        winners = batch.get("winner", [])

        for winner in winners:
            if winner == 'model_a':
                label = 0
            elif winner == 'model_b':
                label = 1
            # If the winner is neither 'model_a' nor 'model_b', you could choose to skip or handle the error here
            else:
                continue  # Or use `label = None` if you want to handle such cases separately
            labels.append(label)

        # Return tokenized output with labels
        return {**tokenized, "labels": labels}

    @staticmethod
    def process_text(text: str) -> str:
        if text is None:
            return ""
        return text.replace("null", "").strip()

In [16]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
train_ds = train_ds.map(encode, batched=True)
eval_ds = eval_ds.map(encode, batched=True)

Map:   0%|          | 0/14535 [00:00<?, ? examples/s]

Map:   0%|          | 0/4845 [00:00<?, ? examples/s]

In [17]:
# train_ds = ds.filter(lambda x: x["fold"] != 0)
# eval_ds = ds.filter(lambda x: x["fold"] == 0)

# Train

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import EvalPrediction

def compute_metrics(eval_pred: EvalPrediction) -> dict:
    # print(eval_pred.predictions)
    # Extract predictions and labels from the EvalPrediction object
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    # print(logits)
    # print("logits", logits[0].shape)
    # print("logits", logits[1].shape)
    # print("labels", labels.shape)
    # Convert logits to predicted labels (assuming binary classification with logits)
    pred_labels = logits.argmax(axis=-1)  # For multi-class, use argmax along the correct axis
    # print(pred_labels.shape)
    # print(labels)
    # Calculate accuracy and other metrics
    accuracy = accuracy_score(labels, pred_labels)

    # Return the metrics as a dictionary
    return {
        "accuracy": accuracy,
        # "precision": precision,
        # "recall": recall,
        # "f1": f1,
    }

In [19]:
training_args = TrainingArguments(
    output_dir=PATH.MODELS,
    overwrite_output_dir=True,
    report_to="tensorboard",
    num_train_epochs=2,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=1,
    eval_strategy="steps",
    eval_steps=0.05,
    save_strategy="steps",
    save_steps=0.05,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
    logging_dir=PATH.LOGS
)


In [20]:
class EvaluateAtEpochEndCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        control.should_evaluate = True

In [ ]:
trainer = Trainer(
    args=training_args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    callbacks=[EvaluateAtEpochEndCallback],
)
trainer.train()

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Accuracy
61,0.939000,1.022747,0.502580
122,0.788900,0.826215,0.503406
183,0.646000,0.750143,0.504231
244,0.748300,0.729250,0.499071


In [ ]:
#  [ 525/5000 07:29 < 1:04:06, 1.16 it/s, Epoch 0.21/2]
# Step	Training Loss	Validation Loss	Accuracy
# 250	0.630700	0.735213	0.497500
# 500	0.769500	0.734566	0.512500